In [ ]:
%load_ext autoreload 
%autoreload 2
import numpy as np
import random
from colossus.cosmology import cosmology
cosmology.setCosmology('planck18')
from colossus.halo import concentration, mass_defs
from tqdm import tqdm
import lensing_estimator
from cosmo import CosmoCalc
import lensing
import foregrounds as fg
import experiments as exp
import sims
import stats
import tools

In [ ]:
nber_clus = 3000
nber_rand = 30000
nber_cov = 500
map_params = [240, 0.25, 240, 0.25]
l, cl = CosmoCalc().cmb_power_spectrum()
cluster = [2e14, 3, 0.7]
l, bl = exp.beam_power_spectrum(beam_fwhm = 1.0)
nl_arr = []
cl_noise_arr = []
noiseval_arr = [0.1, 0.5, 1.0, 3.0]
for noiseval in noiseval_arr:
    l, nl = exp.white_noise_power_spectrum(noiseval_white = noiseval)
    l, nl_deconvolved =exp.white_noise_power_spectrum(noiseval_white = noiseval, beam_fwhm = 1.0)
    nl_arr.append(nl)
    cl_noise_arr.append(nl_deconvolved)
nber_clus_fit = 30000
nber_rand_fit = 30000
mass_int_estimator_comparison = np.arange(0, 4, 0.1)
c200c = 3
z = 0.7

covariance_matrix_estimator_comparison_arr = []
for i, noiseval in enumerate(noiseval_arr):
    covariance_matrix_estimator_comparison, correlation_matrix_estimator_comparison = lensing_estimator.covariance_and_correlation_matrix(nber_cov, nber_clus, map_params, l, cl, bl = bl, nl = nl_arr[i], cl_noise = cl_noise_arr[i])
    covariance_matrix_estimator_comparison_arr.append(covariance_matrix_estimator_comparison)
    with open('/Volumes/Extreme_SSD/codes/master_thesis/code/results/covariance_matrix_estimator_comparison_'+str(noiseval)+'.npy', 'wb') as file:
        np.save(file, covariance_matrix_estimator_comparison)  

model_profiles_estimator_comparison_arr = []   
for i, noiseval in enumerate(noiseval_arr):
    model_profiles_estimator_comparison = lensing_estimator.model_profiles(nber_clus_fit, nber_rand_fit, map_params, l, cl, mass_int_estimator_comparison, c200c, z, bl = bl, cl_noise = cl_noise_arr[i])
    model_profiles_estimator_comparison_arr.append(model_profiles_estimator_comparison)
    with open('/Volumes/Extreme_SSD/codes/master_thesis/code/results/model_profiles_estimator_comparison_'+str(noiseval)+'.npy', 'wb') as file:
        np.save(file, model_profiles_estimator_comparison)    


median_mass_arr, error_arr = [], []
for i in range(len(noiseval_arr)):    
    maps_clus = []
    for j in range(nber_clus):    
        map_clus = sims.cmb_mock_data(map_params, l, cl, cluster = cluster, bl = bl, nl = nl_arr[i])
        maps_clus.append(map_clus)    
    maps_rand = []
    for j in range(nber_rand):    
        map_rand = sims.cmb_mock_data(map_params, l, cl,  bl = bl, nl = nl_arr[i])
        maps_rand.append(map_rand)
    bins, lensing_dipole_profile, stacks = lensing_estimator.lensing_dipole_profile(map_params, maps_clus, maps_rand,  l = l, cl = cl, cl_noise = cl_noise_arr[i])
    data = bins, lensing_dipole_profile, covariance_matrix_estimator_comparison_arr[i]
    likelihood, median_mass, error = stats.run_ml(data, model_profiles_estimator_comparison_arr[i], mass_int_estimator_comparison)
    median_mass_arr.append(median_mass)
    error_arr.append(error)
    
del maps_clus
del maps_rand
    
    
with open('mass_results_estimator_comparison.npy', 'wb') as file:
    np.save(file, median_mass_arr)
    
with open('error_results_estimator_comparison.npy', 'wb') as file:
    np.save(file, error_arr)